[![](https://i.creativecommons.org/l/by-sa/4.0/88x31.png)](http://creativecommons.org/licenses/by-sa/4.0/)

This work is licensed under a [Creative Commons Attribution-ShareAlike
4.0 International
License](http://creativecommons.org/licenses/by-sa/4.0/) 


### About this document 

This document was created using Weave.jl. The code is available in
[on github](https://github.com/schrimpf/NeuralNetworkEconomics.jl). The same
document generates both static webpages and associated [jupyter
notebook](rnn.ipynb). 

$$
\def\indep{\perp\!\!\!\perp}
\def\Er{\mathrm{E}}
\def\R{\mathbb{R}}
\def\En{{\mathbb{E}_n}}
\def\Pr{\mathrm{P}}
\newcommand{\norm}[1]{\left\Vert {#1} \right\Vert}
\newcommand{\abs}[1]{\left\vert {#1} \right\vert}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
$$

In [1]:
markdown = try
  "md" in keys(WEAVE_ARGS) && WEAVE_ARGS["md"]
catch
  false
end

if !("DISPLAY" ∈ keys(ENV))
  # Make gr and pyplot backends for Plots work without a DISPLAY
  ENV["GKSwstype"]="nul"
  ENV["MPLBACKEND"]="Agg"
end
# Make gr backend work with λ and other unicode
ENV["GKS_ENCODING"] = "utf-8"

using NeuralNetworkEconomics
docdir = joinpath(dirname(Base.pathof(NeuralNetworkEconomics)), "..","docs")

using Pkg
Pkg.activate(docdir)
Pkg.instantiate()

  Activating project at `~/.julia/dev/NeuralNetworkEconomics/docs`


In [2]:
using ProgressMeter, JLD2
import HTTP, Gumbo, Cascadia
infile = joinpath(docdir,"jmd","dylanchords.txt")

if !isfile(infile)
  @error "$infile not found. See rnn.jmd for code to create it."  
end
text = String(read(infile));

songs = split(text, "</html>");
songs = songs[1:(end-1)]; # last one is empty
lyrics=Array{String,1}(undef,length(songs));
titles=Array{String,1}(undef,length(songs));

# function to extract text from HTMLNodes
gettext(h::Gumbo.HTMLText) = Gumbo.text(h)
gettext(h::AbstractArray) = length(h)==0 ? "" : prod(gettext, h)
gettext(h::Gumbo.HTMLNode) = gettext(Gumbo.children(h))
# remove chords from verses
function removechords(txt)
  chordregexp=r"(^)( {0,1000}|\()(A|B|C|D|E|G|F)(\S{0,6})(\)| {2,1000}| \.|$).*"m
  txt2=replace(txt, chordregexp => "\n")
  replace(txt2, r"( {2,1000})"=>" ")
end
for (i,song) = enumerate(songs)
  html=Gumbo.parsehtml(song);
  t = eachmatch(Cascadia.Selector(".songtitle"), html.root)
  (length(t)==1) || @warn "multiple songtitles for songs[$i]"
  titles[i] = gettext(t)
  t = eachmatch(Cascadia.Selector(".verse,.refrain"), html.root)  
  lyrics[i] = removechords(gettext(t))
end
lyrics = lyrics[length.(lyrics).>0];

┌ Warning: multiple songtitles for songs[310]
└ @ Main In[2]:28
┌ Warning: multiple songtitles for songs[558]
└ @ Main In[2]:28


In [3]:
using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition
using ProgressMeter

endchar = 'Ω' # any character not in original text
alphabet = [unique(collect(prod(lyrics)))..., endchar]
stop = onehot(endchar, alphabet);
# maxlen = 100 # we will split songs after this many characters
# #batchsize=100 # we use this many "songs" (after splitting) per gradient descent batch
# nextchars = ((x)->[x[2:end]..., endchar]).(lyrics);
# function makex(lyrics, maxlen)
#   hotlyrics=map.(c->onehot(c, alphabet), collect.(lyrics));
#   cs = Int.(ceil.(length.(hotlyrics)./maxlen))
#   foo=chunk.(hotlyrics, cs);
#   X=[gpu.(batchseq(f, stop)) for f in foo];
# end
# Xb = (makex(lyrics, maxlen));
# Yb = (makex(nextchars, maxlen));
# #addaf(daf)
# #  [gpu.(batchseq(map.(c->onehot(c, alphabet), nextchars[p]), stop))
# #     for p in partition(1:length(splitlyrics), batchsize)];
# #Xb = [[x[:,p] for x in X] for p in partition(1:size(X[1],2), batchsize)];
# #Yb = [[y[:,p] for y in Y] for p in partition(1:size(Y[1],2), batchsize)];
# data = collect(zip(Xb,Yb));
# println("There are $(length(data)) batches per epoch")

if true
  text = collect(prod(lyrics));
  hottext = map(ch -> onehot(ch, alphabet), text);
  
  maxlen = 100
  batchsize = 1000
  
  Xseq = collect(partition(gpu.(batchseq(chunk(hottext, batchsize),stop)), maxlen));
  Yseq = collect(partition(gpu.(batchseq(chunk(hottext[2:end], batchsize),stop)), maxlen));
  data = collect(zip(Xseq, Yseq));
end

N = length(alphabet)

function sample(m, alphabet, len)
  m = cpu(m)
  Flux.reset!(m) 
  buf = IOBuffer()
  c = '\n'
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end
opt = RMSProp(0.005)
# this will take awhile, so a fancier call back with a progress meter is nice to have 
function cbgenerator(N, loss, printiter=Int(round(N/10)))
  p=Progress(N, 1, "Training", 25)
  i=0  
  function cb()
    next!(p)
    if (i % printiter==0)
      @show loss()
    end
    i+=1
  end
  return(cb)
end

N = length(alphabet)
testsong = 1
function train_model!(m; N=N, data=data,
                      modelfile="tmp.jld2", 
                      opt=RMSProp(0.01), testset=testsong, epochs=20)
  function loss(xb::V, yb::V) where V <:AbstractVector
    #Flux.reset!(m)
    l = sum(crossentropy.(m.(xb),yb))/length(xb)
    #l = crossentropy(m(xb),yb)
    Flux.truncate!(m)
    return(l)
  end
  #function loss(xb, yb)
  #  Flux.reset!(m) # after each song, forget gradients  
  #  l = crossentropy(m(xb),yb)
  #  return(l)
  #end
    
  cb=cbgenerator(length(data)*(epochs+1),
                 ()->loss(data[testset]...)) 

  if isfile(modelfile)
    @load modelfile cpum
    m = gpu(cpum)
  end
  
  @time Flux.train!(loss, Flux.params(m), data, opt, cb = cb)
  println("Sampling after 1 epoch:")
  sample(m, alphabet, 1000) |> println
    
  Flux.@epochs epochs Flux.train!(loss, Flux.params(m), data, opt,
                                  cb = cb)
  cpum = cpu(m)
  @save modelfile cpum
  return(m)
end

for L in [75] #, 32, 64, 128] #, 256, 512]
  #L = 100
  file = joinpath(docdir,"jmd","dylanlyrics-$L.jld2")
  m = Chain(LSTM(N, L), LSTM(L, L),  Dense(L, N),  softmax) |> gpu
  #m = Chain(GRU(N, L), GRU(L, L), Dense(L, N),  softmax) |> gpu
  opt=RMSProp(0.01)
  m = train_model!(m, opt=opt, epochs=100, modelfile=file)
  println("ΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞ")
  println("ΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞ")
  println("ΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞ")  
  println("Model $L has $(sum([prod(size(p)) for p in Flux.params(m)])) parameters")
  println("Sample from model $L")
  println("ΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞΞ")
  println(sample(m, alphabet, 5000))
  println()
end

LoadError: UndefVarError: truncate! not defined

# References